# Ho-Lee model

> Get data from term-structure

In [1]:
from zero import zeros, forward_rate
import pandas as pd

_bond_data = [
    [100, 0.50, 0, 99.1338],
    [100, 1.00, 0, 97.8925],
    [100, 1.50, 0, 96.1531]
]

# calculate zero coupon bond's yield
_yield = list(map(lambda x: round(x, 4), zeros(_bond_data)))
print("Yield: " + str(_yield))
# calculate forward rate
_forward = list(map(lambda x: round(x, 4), forward_rate(_bond_data, 0.5)))
print("Forward Rate: " + str(_forward))
print()

_market_data = pd.DataFrame(
    {
        "Maturity": [0.5, 1.0, 1.5],
        "Price": [99.1338, 97.8925, 96.1531],
        "Yield": _yield,
        "Forward Rate": _forward
    }
)

mp0 = _market_data.Price[0]
mp1 = _market_data.Price[1]
mp2 = _market_data.Price[2]

print(_market_data)

Yield: [0.0174, 0.0213, 0.0262]
Forward Rate: [0.0174, 0.0252, 0.0359]

   Maturity    Price   Yield  Forward Rate
0       0.5  99.1338  0.0174        0.0174
1       1.0  97.8925  0.0213        0.0252
2       1.5  96.1531  0.0262        0.0359


> Get irm when theta0 = 0.01

In [2]:
import binomial as bi
from price import round_model

r00 = 0.0174
theta0 = 0.01
theta1 = 0.00
delta = 0.5
vol = 0.0173

r10 = r00 + theta0*delta + vol*delta**0.5
r11 = r00 + theta0*delta - vol*delta**0.5

print("Interest rate going up: " + str(round(r10*100, 2)) + '%')
print("Interest rate going down: " + str(round(r11*100, 2)) + '%')

irm = bi.model([[r00], [r10, r11]])
round_model(irm, 4)
print("Interest rate model:")
print(irm.beautify())

Interest rate going up: 3.46%
Interest rate going down: 1.02%
Interest rate model:
| 0.0174 | 0.0346 | 
         | 0.0102 | 


> Get bond-price model by interest rate model with theta=0.01

In [3]:
from price import bond_model

bpm = round_model(bond_model(100, irm, [0.5], 0.5), 4)
print("Bond price model:")
print(bpm.beautify())

Bond price model:
| 98.0315 | 98.2849 | 100 | 
          | 99.4913 | 100 | 
                 | 100 | 


> Calibrate theta

In [4]:
from scipy.optimize import fsolve

ir_by_ho_lee = lambda ir, theta, vol, delta: [ir+theta*delta + vol*delta**0.5, ir+theta*delta - vol*delta**0.5]
get_irm = lambda th: bi.model([[r00], ir_by_ho_lee(r00, th, vol, delta)])
get_bpm = lambda th: bond_model(100, get_irm(th), [0.5], 0.5)
price_error = lambda th: (get_bpm(th).value(0, 0)-mp1)**2

_result = fsolve(price_error, theta0)
print("Get result: " + str(_result))

theta0 = _result[0]
print("theta0: " + str(round(theta0*100, 4)) + '%')
print()

irm1 = get_irm(theta0).change_model(round, 4)
print("Interest rate model:")
print(irm1.beautify())
print()

bpm1 = get_bpm(theta0).change_model(round, 4)
print("Bond price model:")
print(bpm1.beautify())

Get result: [0.01567581]
theta0: 1.5676%

Interest rate model:
| 0.0174 | 0.0375 | 
         | 0.013 | 

Bond price model:
| 97.8925 | 98.1439 | 100 | 
          | 99.3519 | 100 | 
                 | 100 | 


> Increase interest rate tree model

In [5]:
def _get_next_irm(__irm, __th, __vol, __delta):
    next_irm_data = []
    next_data = []
    for i in __irm.data[-1]:
        for j in ir_by_ho_lee(i, __th, __vol, __delta):
            next_data.append(j)
    for i in __irm.data:
        next_irm_data.append(i)
    next_irm_data.append(next_data)
    return bi.model(next_irm_data)

get_bpm = lambda irm, th, vol, delta: bond_model(100, _get_next_irm(irm, th, vol, delta), [0.5, 0.5], 0.5)
price_error = lambda th: (get_bpm(irm1, th, vol, delta).value(0, 0)-mp2)**2

_result = fsolve(price_error, theta1)
print("Get result: " + str(_result))

theta1 = _result[0]
print("theta1: " + str(round(theta1*100, 4)) + '%')
print()

irm2 = round_model(_get_next_irm(irm1, theta1, vol, delta), 4)
print("irm:")
print(irm2.beautify())
print()

bpm2 = round_model(bond_model(100, irm2, [0.5, 0.5], 0.5),4)
print("bpm:")
print(bpm2.beautify())

Get result: [0.02148883]
theta1: 2.1489%

irm:
| 0.0174 | 0.0375 | 0.0605 | 
         | 0.013 | 0.036 | 
              | 0.0115 | 

bpm:
| 96.1528 | 95.8049 | 97.0203 | 100 | 
          | 98.1811 | 98.2161 | 100 | 
                 | 99.4266 | 100 | 
                        | 100 | 


# BDT-model

> Use price.ho_lee_irm, the result of ho_lee irm is more simple

In [6]:
from price import ho_lee_irm

irm_with_holee = ho_lee_irm(100, 0.01, [98.3], 0.0121, 0.5).change_model(round, 3)

print("Solve irm with price.ho_lee_irm:")
print(irm_with_holee.beautify())

Solve irm with price.ho_lee_irm:
| 0.01 | 0.033 | 
       | 0.016 | 


> BDT model

In [7]:
import numpy as np
from scipy.optimize import minimize

log_vol = 0.2142

irm_list = [[r00]]
bdt_step = lambda vol, delta: vol*delta**0.5
highest_step = lambda ir, th, vol, delta: np.log(ir)+th*delta+bdt_step(vol, delta)

def _get_next_irl(__irm_list, __th, __vol, __delta):
    _irl = list(map(lambda x: x, __irm_list))
    next_data = []
    _length = len(_irl[-1])
    _first = _irl[-1][0]
    s = float(highest_step(_first, __th, __vol, __delta))
    next_data.append(np.exp(s))
    i = 1
    while i < _length+1:
        s -= 2*bdt_step(__vol, __delta)
        next_data.append(np.exp(s))
        i += 1
    _irl.append(next_data)
    
    return _irl

cnt = 1
for i in [mp1, mp2]:
    _rnp = []
    for k in range(cnt):
        _rnp.append(0.5)
    price_error = lambda th: (bond_model(100, bi.model(_get_next_irl(irm_list, th, vol, delta), simplify=True), _rnp, 0.5).value(0, 0)-i)**2
    _result = minimize(price_error, 0.01).x
    print(f"theta{cnt-1}: " + str(_result))
    print()
    
    theta = _result[0]
    irm_list = _get_next_irl(irm_list, theta, vol, delta)
    bdt_irm = bi.model(irm_list, simplify=True).change_model(round, 4)
    print("BDT interest rate model:")
    print(bdt_irm.beautify())
    print()
    
    print("BDT bond price model:")
    print(bond_model(100, bdt_irm, _rnp, 0.5).change_model(round, 4).beautify())
    print(), print()
    cnt += 1

theta0: [0.74063862]

BDT interest rate model:
| 0.0174 | 0.0255 | 
         | 0.0249 | 

BDT bond price model:
| 97.8925 | 98.7331 | 100 | 
          | 98.7627 | 100 | 
                 | 100 | 


theta1: [0.7058947]

BDT interest rate model:
| 0.0174 | 0.0255 | 0.0367 | 
         | 0.0249 | 0.0359 | 
               | 0.035 | 

BDT bond price model:
| 96.1522 | 96.9573 | 98.1817 | 100 | 
          | 97.0276 | 98.221 | 100 | 
                 | 98.2652 | 100 | 
                        | 100 | 


